## 准备数据

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [7]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [8]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([28 * 28, 256]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([256]), dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([256, 10]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), dtype=tf.float32)

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 28 * 28])  # 展平输入图像
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  # 第一层 ReLU 激活
        logits = tf.matmul(h1, self.W2) + self.b2  # 输出层 (未归一化的logits)

        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [9]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 147.01784 ; accuracy 0.1294
epoch 1 : loss 129.44061 ; accuracy 0.13258334
epoch 2 : loss 117.5348 ; accuracy 0.13425
epoch 3 : loss 108.79885 ; accuracy 0.1357
epoch 4 : loss 102.06229 ; accuracy 0.13611667
epoch 5 : loss 96.7385 ; accuracy 0.1375
epoch 6 : loss 92.39063 ; accuracy 0.13928333
epoch 7 : loss 88.7524 ; accuracy 0.14093333
epoch 8 : loss 85.64228 ; accuracy 0.1429
epoch 9 : loss 82.90957 ; accuracy 0.14681667
epoch 10 : loss 80.444084 ; accuracy 0.15145
epoch 11 : loss 78.17881 ; accuracy 0.15681666
epoch 12 : loss 76.062775 ; accuracy 0.16216667
epoch 13 : loss 74.06458 ; accuracy 0.16801667
epoch 14 : loss 72.167595 ; accuracy 0.17331667
epoch 15 : loss 70.3595 ; accuracy 0.18028334
epoch 16 : loss 68.63015 ; accuracy 0.18663333
epoch 17 : loss 66.97259 ; accuracy 0.19295
epoch 18 : loss 65.38364 ; accuracy 0.20041667
epoch 19 : loss 63.859367 ; accuracy 0.20736666
epoch 20 : loss 62.395775 ; accuracy 0.2149
epoch 21 : loss 60.991165 ; accuracy 0.2224333